### Config

In [ ]:
import os, json
import pandas as pd

path_to_json = '/content/drive/MyDrive/hackathon-nlp/text-json'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)

['aoe4day9.json', 'its happening again.json', 'build a dock.json', 'blinking.json', 'bad fences.json', 'can somebody clip this.json', 'cant click on villagers.json', 'LOS problem.json', "repair doesn't cost wood.json", 'took my buff.json', 'donut hitbox.json', 'hit boxes.json', 'healing from the inside.json', 'this means fish.json', 'landmark bug.json', 'tax collector.json', 'theres the bug.json', 'fishing ship.json', 'the pathing.json', 'mongol town center.json', 'invisible wall.json', 'FOLDER LABEL.json', 'mehter buff.json', 'repair interrupt.json', 'mangonel.json', 'cursor action reset.json', 'double upgrade.json', 'horse selection.json', 'see oponent.json', 'they dont build.json']


In [ ]:
pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["MODEL_DIR"] = '../model'

In [ ]:
jsons_dataframe = pd.DataFrame(columns=['timestamp', 'duration', 'text', 'bug', 'aug'])

In [ ]:
columns_jsons_dataframe=['timestamp', 'duration', 'text', 'bug', 'aug']

### Read dataset in json and transfrom to dataframe

In [ ]:
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
      #print(json_file)
      json_texts = json.load(json_file)
      for json_text in json_texts:
        timestamp = json_text['timestamp']
        duration = json_text['duration']
        text = json_text['text']
        bug = json_text['bug']
        aug = 'False'
        df = pd.DataFrame([[timestamp, duration, text, bug, aug]], columns=columns_jsons_dataframe)
        jsons_dataframe = jsons_dataframe.append(df, ignore_index=True)
print(jsons_dataframe)

     timestamp  duration                                               text  \
0         0.06      3.39                     formations? sure. lets make a.   
1         4.14      1.37                       eric thing i can experiment.   
2         8.92      8.97  oh yeah and theres also so many weirdness issu...   
3        19.07     12.74  so it ignores anything. let me try this again....   
4        32.89      1.37                      it permits me to build right?   
..         ...       ...                                                ...   
203      16.87      0.87                         just tell him to build it.   
204      18.66      0.83                                there you go. look.   
205      20.64      9.03  the hp is not going any faster. they just stop...   
206      33.58      1.59                     but theyre already doing that.   
207      38.62      1.95               now shes got to come help all right?   

    bug    aug  
0     0  False  
1     0  False  


In [ ]:
positive_df = jsons_dataframe[jsons_dataframe['bug'] == 1]
print(positive_df)

     timestamp  duration                                               text  \
2         8.92      8.97  oh yeah and theres also so many weirdness issu...   
3        19.07     12.74  so it ignores anything. let me try this again....   
9         4.52      3.17  no no guys its the bug again. can someone clip...   
16        7.20      2.72         ohh my fucking god build the fucking dock.   
18       13.92      9.18  i ive no idea why its so hard for this game to...   
..         ...       ...                                                ...   
190      70.20      2.22                  might get hotfixed yeah tomorrow.   
196      20.23      7.18  oh the game just literally never executed the ...   
197      28.40     10.63  and the right clicking on things has been incr...   
200       0.05     20.73  before the celts among dlc. true hi view. alri...   
205      20.64      9.03  the hp is not going any faster. they just stop...   

    bug    aug  
2     1  False  
3     1  False  


### Data Augmentation
#### Augment the postive cases dataset through Back Translatoin and Contextual Word Embeddings

In [ ]:
import nlpaug.augmenter.word as naw
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)

In [ ]:
aug = naw.ContextualWordEmbsAug(
  model_path='bert-base-uncased', action="insert")

for _, pos_case in positive_df.iterrows():
  text = pos_case['text']  
  augmented_text = aug.augment(text)
  back_aug = back_translation_aug.augment(text)
  #print("Original:")
  #print(text)
  #print("Augmented Text:")
  #print(augmented_text[0])
  df_aug = pd.DataFrame([[0, 0, augmented_text[0], 1, "True"]], columns=columns_jsons_dataframe)
  df_back_aug = pd.DataFrame([[0, 0, back_aug[0], 1, "True"]], columns=columns_jsons_dataframe)
  positive_df = positive_df.append(df_aug, ignore_index=True)
  positive_df = positive_df.append(df_back_aug, ignore_index=True)


In [ ]:
positive_df[positive_df['aug'] == "True"]

,timestamp,duration,text,bug,aug
69,0.0,0.0,oh yeah sure and theres also be so many minor ...,1,True
70,0.0,0.0,"Oh yes, and there are also so many oddities th...",1,True
71,0.0,0.0,so it ignores these anything. let me try this ...,1,True
72,0.0,0.0,so it ignores anything. Let me give it another...,1,True
73,0.0,0.0,no names no guys mike its on the bug again. ca...,1,True
...,...,...,...,...,...
202,0.0,0.0,and the right click on things was incredibly f...,1,True
203,0.0,0.0,before the fall celts among dlc. true hi to vi...,1,True
204,0.0,0.0,before the celts under dlc. true hello view. w...,1,True
205,0.0,0.0,the hp is not going any faster. they just stop...,1,True


In [ ]:
positive_df

,timestamp,duration,text,bug,aug
0,8.92,8.97,oh yeah and theres also so many weirdness issu...,1,False
1,19.07,12.74,so it ignores anything. let me try this again....,1,False
2,4.52,3.17,no no guys its the bug again. can someone clip...,1,False
3,7.20,2.72,ohh my fucking god build the fucking dock.,1,False
4,13.92,9.18,i ive no idea why its so hard for this game to...,1,False
...,...,...,...,...,...
202,0.00,0.00,and the right click on things was incredibly f...,1,True
203,0.00,0.00,before the fall celts among dlc. true hi to vi...,1,True
204,0.00,0.00,before the celts under dlc. true hello view. w...,1,True
205,0.00,0.00,the hp is not going any faster. they just stop...,1,True


In [ ]:
jsons_dataframe = jsons_dataframe.append(positive_df, ignore_index=True)

In [ ]:
jsons_dataframe

,timestamp,duration,text,bug,aug
0,0.06,3.39,formations? sure. lets make a.,0,False
1,4.14,1.37,eric thing i can experiment.,0,False
2,8.92,8.97,oh yeah and theres also so many weirdness issu...,1,False
3,19.07,12.74,so it ignores anything. let me try this again....,1,False
4,32.89,1.37,it permits me to build right?,0,False
...,...,...,...,...,...
410,0.00,0.00,and the right click on things was incredibly f...,1,True
411,0.00,0.00,before the fall celts among dlc. true hi to vi...,1,True
412,0.00,0.00,before the celts under dlc. true hello view. w...,1,True
413,0.00,0.00,the hp is not going any faster. they just stop...,1,True


In [ ]:
jsons_dataframe


,timestamp,duration,text,bug,aug
0,0.06,3.39,formations? sure. lets make a.,0,False
1,4.14,1.37,eric thing i can experiment.,0,False
2,8.92,8.97,oh yeah and theres also so many weirdness issu...,1,False
3,19.07,12.74,so it ignores anything. let me try this again....,1,False
4,32.89,1.37,it permits me to build right?,0,False
...,...,...,...,...,...
410,0.00,0.00,and the right click on things was incredibly f...,1,True
411,0.00,0.00,before the fall celts among dlc. true hi to vi...,1,True
412,0.00,0.00,before the celts under dlc. true hello view. w...,1,True
413,0.00,0.00,the hp is not going any faster. they just stop...,1,True


In [ ]:
result = jsons_dataframe.to_json('df.json', orient="records")
result